In [1]:
!pip install nibabel
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import nibabel as nib
import glob
import os
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
import torchmetrics
from torchmetrics import Dice

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [3]:
TRAIN_DATASET_PATH = "./Data/kits19/data"
VALIDATION_DATASET_PATH = "./Data/kits19/data"

In [5]:
img_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*imaging.nii.gz'))
seg_list = sorted(glob.glob(TRAIN_DATASET_PATH+'/*/*segmentation.nii.gz'))
print(len(img_list))

300


In [6]:
from torch import nn
import torch
import torch.nn.functional as F


class CNA3d(nn.Module): # conv + norm + activation
    def __init__(self, in_channels, out_channels, kSize, stride, padding=(1,1,1), bias=True, norm_args=None, activation_args=None):
        super().__init__()
        self.norm_args = norm_args
        self.activation_args = activation_args
        
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kSize, stride=stride, padding=padding, bias=bias)

        if norm_args is not None:
            self.norm = nn.InstanceNorm3d(out_channels, **norm_args)

        if activation_args is not None:
            self.activation = nn.LeakyReLU(**activation_args)


    def forward(self, x):
        x = self.conv(x)

        if self.norm_args is not None:
            x = self.norm(x)

        if self.activation_args is not None:
            x = self.activation(x)
        return x



class CB3d(nn.Module): # conv block 3d
    def __init__(self, in_channels, out_channels, kSize=(3,3), stride=(1,1), padding=(1,1,1), bias=True,
                 norm_args:tuple=(None,None), activation_args:tuple=(None,None)):
        super().__init__()

        self.conv1 = CNA3d(in_channels, out_channels, kSize=kSize[0], stride=stride[0],
                             padding=padding, bias=bias, norm_args=norm_args[0], activation_args=activation_args[0])

        self.conv2 = CNA3d(out_channels, out_channels,kSize=kSize[1], stride=stride[1],
                             padding=padding, bias=bias, norm_args=norm_args[1], activation_args=activation_args[1])

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x





class BasicNet(nn.Module):
    norm_kwargs = {'affine': True}
    activation_kwargs = {'negative_slope': 1e-2, 'inplace': True}

    def __init__(self):
        super(BasicNet, self).__init__()

    def parameter_count(self):
        print("model have {} paramerters in total".format(sum(x.numel() for x in self.parameters()) / 1e6))

In [7]:
from torch import nn
#from basic_module import *


def FMU(x1, x2, mode='sub'):
    """
    feature merging unit
    Args:
        x1:
        x2:
        mode: type of fusion
    Returns:
    """
    if mode == 'sum':
        return torch.add(x1, x2)
    elif mode == 'sub':
        return torch.abs(x1 - x2)
    elif mode == 'cat':
        return torch.cat((x1, x2), dim=1)
    else:
        raise Exception('Unexpected mode')


class Down(BasicNet):
    def __init__(self, in_channels, out_channels, mode: tuple, FMU='sub', downsample=True, min_z=8):
        """
        basic module at downsampling stage
        Args:
            in_channels:
            out_channels:
            mode: represent the streams coming in and out. e.g., ('2d', 'both'): one input stream (2d) and two output streams (2d and 3d)
            FMU: determine the type of feature fusion if there are two input streams
            downsample: determine whether to downsample input features (only the first module of MNet do not downsample)
            min_z: if the size of z-axis < min_z, maxpooling won't be applied along z-axis
        """
        super().__init__()
        self.mode_in, self.mode_out = mode
        self.downsample = downsample
        self.FMU = FMU
        self.min_z = min_z
        norm_args = (self.norm_kwargs, self.norm_kwargs)
        activation_args = (self.activation_kwargs, self.activation_kwargs)

        if self.mode_out == '2d' or self.mode_out == 'both':
            self.CB2d = CB3d(in_channels=in_channels, out_channels=out_channels,
                             kSize=((1, 3, 3), (1, 3, 3)), stride=(1, 1), padding=(0, 1, 1),
                             norm_args=norm_args, activation_args=activation_args)

        if self.mode_out == '3d' or self.mode_out == 'both':
            self.CB3d = CB3d(in_channels=in_channels, out_channels=out_channels,
                             kSize=(3, 3), stride=(1, 1), padding=(1, 1, 1),
                             norm_args=norm_args, activation_args=activation_args)

    def forward(self, x):
        if self.downsample:
            if self.mode_in == 'both':
                x2d, x3d = x
                p2d = F.max_pool3d(x2d, kernel_size=(1, 2, 2), stride=(1, 2, 2))
                if x3d.shape[2] >= self.min_z:
                    p3d = F.max_pool3d(x3d, kernel_size=(2, 2, 2), stride=(2, 2, 2))
                else:
                    p3d = F.max_pool3d(x3d, kernel_size=(1, 2, 2), stride=(1, 2, 2))

                x = FMU(p2d, p3d, mode=self.FMU)

            elif self.mode_in == '2d':
                x = F.max_pool3d(x, kernel_size=(1, 2, 2), stride=(1, 2, 2))

            elif self.mode_in == '3d':
                if x.shape[2] >= self.min_z:
                    x = F.max_pool3d(x, kernel_size=(2, 2, 2), stride=(2, 2, 2))
                else:
                    x = F.max_pool3d(x, kernel_size=(1, 2, 2), stride=(1, 2, 2))

        if self.mode_out == '2d':
            return self.CB2d(x)
        elif self.mode_out == '3d':
            return self.CB3d(x)
        elif self.mode_out == 'both':
            return self.CB2d(x), self.CB3d(x)



class Up(BasicNet):
    def __init__(self, in_channels, out_channels, mode: tuple, FMU='sub'):
        """
        basic module at upsampling stage
        Args:
            in_channels:
            out_channels:
            mode: represent the streams coming in and out. e.g., ('2d', 'both'): one input stream (2d) and two output streams (2d and 3d)
            FMU: determine the type of feature fusion if there are two input streams
        """
        super().__init__()
        self.mode_in, self.mode_out = mode
        self.FMU = FMU
        norm_args = (self.norm_kwargs, self.norm_kwargs)
        activation_args = (self.activation_kwargs, self.activation_kwargs)

        if self.mode_out == '2d' or self.mode_out == 'both':
            self.CB2d = CB3d(in_channels=in_channels, out_channels=out_channels,
                             kSize=((1, 3, 3), (1, 3, 3)), stride=(1, 1), padding=(0, 1, 1),
                             norm_args=norm_args, activation_args=activation_args)

        if self.mode_out == '3d' or self.mode_out == 'both':
            self.CB3d = CB3d(in_channels=in_channels, out_channels=out_channels,
                             kSize=(3, 3), stride=(1, 1), padding=(1, 1, 1),
                             norm_args=norm_args, activation_args=activation_args)

    def forward(self, x):
        x2d, xskip2d, x3d, xskip3d = x

        tarSize = xskip2d.shape[2:]
        up2d = F.interpolate(x2d, size=tarSize, mode='trilinear', align_corners=False)
        up3d = F.interpolate(x3d, size=tarSize, mode='trilinear', align_corners=False)

        cat = torch.cat([FMU(xskip2d, xskip3d, self.FMU), FMU(up2d, up3d, self.FMU)], dim=1)

        if self.mode_out == '2d':
            return self.CB2d(cat)
        elif self.mode_out == '3d':
            return self.CB3d(cat)
        elif self.mode_out == 'both':
            return self.CB2d(cat), self.CB3d(cat)



class MNet(BasicNet):
    def __init__(self, in_channels, num_classes, kn=(32, 48, 64, 80, 96), ds=True, FMU='sub'):
        """
        Args:
            in_channels: channels of input
            num_classes: output classes
            kn: the number of kernels
            ds: deep supervision
            FMU: type of feature merging unit
        """
        super().__init__()
        self.ds = ds
        self.num_classes = num_classes

        channel_factor = {'sum': 1, 'sub': 1, 'cat': 2}
        fct = channel_factor[FMU]

        self.down11 = Down(in_channels, kn[0], ('/', 'both'), downsample=False)
        self.down12 = Down(kn[0], kn[1], ('2d', 'both'))
        self.down13 = Down(kn[1], kn[2], ('2d', 'both'))
        self.down14 = Down(kn[2], kn[3], ('2d', 'both'))
        self.bottleneck1 = Down(kn[3], kn[4], ('2d', '2d'))
        self.up11 = Up(fct * (kn[3] + kn[4]), kn[3], ('both', '2d'), FMU)
        self.up12 = Up(fct * (kn[2] + kn[3]), kn[2], ('both', '2d'), FMU)
        self.up13 = Up(fct * (kn[1] + kn[2]), kn[1], ('both', '2d'), FMU)
        self.up14 = Up(fct * (kn[0] + kn[1]), kn[0], ('both', 'both'), FMU)

        self.down21 = Down(kn[0], kn[1], ('3d', 'both'))
        self.down22 = Down(fct * kn[1], kn[2], ('both', 'both'), FMU)
        self.down23 = Down(fct * kn[2], kn[3], ('both', 'both'), FMU)
        self.bottleneck2 = Down(fct * kn[3], kn[4], ('both', 'both'), FMU)
        self.up21 = Up(fct * (kn[3] + kn[4]), kn[3], ('both', 'both'), FMU)
        self.up22 = Up(fct * (kn[2] + kn[3]), kn[2], ('both', 'both'), FMU)
        self.up23 = Up(fct * (kn[1] + kn[2]), kn[1], ('both', '3d'), FMU)

        self.down31 = Down(kn[1], kn[2], ('3d', 'both'))
        self.down32 = Down(fct * kn[2], kn[3], ('both', 'both'), FMU)
        self.bottleneck3 = Down(fct * kn[3], kn[4], ('both', 'both'), FMU)
        self.up31 = Up(fct * (kn[3] + kn[4]), kn[3], ('both', 'both'), FMU)
        self.up32 = Up(fct * (kn[2] + kn[3]), kn[2], ('both', '3d'), FMU)

        self.down41 = Down(kn[2], kn[3], ('3d', 'both'), FMU)
        self.bottleneck4 = Down(fct * kn[3], kn[4], ('both', 'both'), FMU)
        self.up41 = Up(fct * (kn[3] + kn[4]), kn[3], ('both', '3d'), FMU)

        self.bottleneck5 = Down(kn[3], kn[4], ('3d', '3d'))


        self.outputs = nn.ModuleList(
            [nn.Conv3d(c, num_classes, kernel_size=(1, 1, 1), stride=1, padding=0, bias=False)
             for c in [kn[0], kn[1], kn[1], kn[2], kn[2], kn[3], kn[3]]]
        )

    def forward(self, x):
        down11 = self.down11(x)
        down12 = self.down12(down11[0])
        down13 = self.down13(down12[0])
        down14 = self.down14(down13[0])
        bottleNeck1 = self.bottleneck1(down14[0])

        down21 = self.down21(down11[1])
        down22 = self.down22([down21[0], down12[1]])
        down23 = self.down23([down22[0], down13[1]])
        bottleNeck2 = self.bottleneck2([down23[0], down14[1]])

        down31 = self.down31(down21[1])
        down32 = self.down32([down31[0], down22[1]])
        bottleNeck3 = self.bottleneck3([down32[0], down23[1]])

        down41 = self.down41(down31[1])
        bottleNeck4 = self.bottleneck4([down41[0], down32[1]])

        bottleNeck5 = self.bottleneck5(down41[1])

        up41 = self.up41([bottleNeck4[0], down41[0], bottleNeck5, down41[1]])

        up31 = self.up31([bottleNeck3[0], down32[0], bottleNeck4[1], down32[1]])
        up32 = self.up32([up31[0], down31[0], up41, down31[1]])

        up21 = self.up21([bottleNeck2[0], down23[0], bottleNeck3[1], down23[1]])
        up22 = self.up22([up21[0], down22[0], up31[1], down22[1]])
        up23 = self.up23([up22[0], down21[0], up32, down21[1]])
        
        up11 = self.up11([bottleNeck1, down14[0], bottleNeck2[1], down14[1]])
        up12 = self.up12([up11, down13[0], up21[1], down13[1]])
        up13 = self.up13([up12, down12[0], up22[1], down12[1]])
        up14 = self.up14([up13, down11[0], up23, down11[1]])


        if self.ds:
            features = [up14[0]+up14[1], up23, up13, up32, up12, up41,up11]
            return [self.outputs[i](features[i]) for i in range(7)]
        else:
            return self.outputs[0](up14[0]+up14[1])



if __name__ == '__main__':
    mn = MNet(1, 3, kn=(2, 2, 2, 2, 2), ds=True, FMU='sub')
    input = torch.randn((1, 1, 19, 255,256))
    output = mn(input)

    print([e.shape for e in output])

[torch.Size([1, 3, 19, 255, 256]), torch.Size([1, 3, 9, 127, 128]), torch.Size([1, 3, 19, 127, 128]), torch.Size([1, 3, 4, 63, 64]), torch.Size([1, 3, 19, 63, 64]), torch.Size([1, 3, 4, 31, 32]), torch.Size([1, 3, 19, 31, 32])]


In [8]:
from torch.utils.data import Dataset

class KitsDataset(Dataset):
    def __init__(self, img, seg):
        '''
        TODO: Store `seqs`. to `self.x` and `hfs` to `self.y`.

        Note that you DO NOT need to covert them to tensor as we will do this later.
        Do NOT permute the data.
        '''
        # your code here
        #raise NotImplementedError
    
        self.img = img
        self.seg = seg
    
    def __len__(self):
        '''
        TODO: Return the number of samples (i.e. patients).
        '''
        # your code here
        #raise NotImplementedError
        return len(self.img)

    def __getitem__(self, index):
        '''
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        '''
        return self.img[index], self.seg[index]



NameError: name 'model' is not defined

In [10]:
im_list = []
sg_list = []

for i in range(10,15):#range(len(t2_list)):
    print(img_list[i])
    temp_img = nib.load(img_list[i]).get_fdata()
    temp_img = torch.from_numpy(scaler.fit_transform(temp_img.reshape(-1,temp_img.shape[-1])).reshape(temp_img.shape))
    #print(temp_img.shape)
    #print(temp_img)
    temp_seg = nib.load(seg_list[i]).get_fdata()
    temp_seg = torch.from_numpy(scaler.fit_transform(temp_seg.reshape(-1,temp_seg.shape[-1])).reshape(temp_seg.shape))
    
    im_list.append(temp_img)
    sg_list.append(temp_seg)
    '''
    temp_t2 = nib.load(t2_list[i]).get_fdata()
    temp_t2 = scaler.fit_transform(temp_t2.reshape(-1,temp_t2.shape[-1])).reshape(temp_t2.shape)
    
    temp_t1=nib.load(t1ce_list[i]).get_fdata()
    temp_t1=scaler.fit_transform(temp_t1.reshape(-1, temp_t1.shape[-1])).reshape(temp_t1.shape)
   
    temp_flair=nib.load(flair_list[i]).get_fdata()
    temp_flair=scaler.fit_transform(temp_flair.reshape(-1, temp_flair.shape[-1])).reshape(temp_flair.shape)
        
    temp_mask=nib.load(mask_list[i]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3 
    
    t2_img_list.append(temp_t2)
    t1_img_list.append(temp_t1)
    flair_img_list.append(temp_flair)
    mask_img_list.append(temp_mask)
    '''
kits_dataset = KitsDataset(im_list, sg_list)

./Data/kits19/data/case_00010/imaging.nii.gz
./Data/kits19/data/case_00011/imaging.nii.gz
./Data/kits19/data/case_00012/imaging.nii.gz
./Data/kits19/data/case_00013/imaging.nii.gz
./Data/kits19/data/case_00014/imaging.nii.gz


In [35]:
from torch.utils.data import DataLoader
dataloader = DataLoader(kits_dataset,batch_size=1)
mn = MNet(1, 3, kn=(2, 2, 2, 2, 2), ds=False, FMU='sub').float()
criterion = torchmetrics.Dice()
optimizer = torch.optim.SGD(mn.parameters(), lr=0.001) 
assert isinstance(optimizer, torch.optim.SGD)


mn.train()
for (img,mask) in dataloader: 
    #print(type(t2))
    #print(torch.unsqueeze(t2.float(),dim=0).size())
    #print(img)
    optimizer.zero_grad()
    output = mn(torch.unsqueeze(img.float(),dim=0))
    pred = torch.unsqueeze(torch.sum(output[0],dim=0),dim=0)
    #print(len(output))
    #print(torch.unsqueeze(torch.sum(output[0],dim=0),dim=0).size())
    #print(mask.size())
    #print(mask[mask>0])
    print(pred)
    pred[pred<0] = 0
    print(output[0][output[0]<0])
    #print(mask)
    loss = criterion(pred,mask.int())
    loss.backward()
    optimizer.step()
    #print(output)

tensor([[[[-3.7695e+00, -3.9521e+00, -4.7775e+00,  ..., -2.3302e-01,
           -1.0149e+00, -2.4387e+00],
          [ 9.3621e-01, -5.5647e-04, -1.0856e+00,  ...,  1.3083e+00,
            1.9675e-01, -9.5304e-01],
          [-8.3016e-02, -1.5582e-01, -1.4258e+00,  ...,  1.3259e+00,
           -6.2103e-02, -1.7839e+00],
          ...,
          [ 4.4792e-01,  1.9461e-02,  1.3964e+00,  ...,  1.3099e+00,
            5.3495e-01, -1.1274e-01],
          [-6.9509e-01, -9.5879e-01, -5.0957e-01,  ..., -9.4747e-01,
           -1.9981e+00, -3.2027e-01],
          [ 4.2758e-01,  3.0260e-01,  1.1658e+00,  ...,  1.1104e+00,
            3.9253e-01,  3.2506e-01]],

         [[-3.2354e+00, -4.1189e+00, -4.8267e+00,  ..., -1.1367e+00,
           -4.2334e-01,  3.5502e-01],
          [ 2.5624e-01,  1.1074e+00,  1.0721e+00,  ...,  6.5080e-01,
           -4.3853e-02,  5.0463e-01],
          [-4.4199e-02,  5.9178e-02, -8.6187e-03,  ...,  5.9317e-02,
           -9.6594e-01,  9.0489e-02],
          ...,
     

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn